<a href="https://colab.research.google.com/github/takao-takenouchi/dp_tutorial/blob/main/DP_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Differential Privacyの基本

# はじめに

本チュートリアルはPyDPを利用し、Differential Privacyの概要の理解を目指します。PyDPとは、OpenMindが開発しているApache License 2.0で利用可能なOSSライブラリで、Google's Differential Privacy libraryをサポートし、Laplaceノイズに対応し、整数と浮動小数点に対応しています。詳細は、PyDPのサイトを参照ください。
https://pydp.readthedocs.io/en/latest/index.html

本チュートリアルは、以下を参考に作成しています。
*   PyDPのチュートリアル
    * OM PriCon2020 Tutorial: Differential Privacy Using PyDP - Chinmay Shah
        * https://www.youtube.com/watch?v=15OgnNsvEo8
        * https://github.com/google/differential-privacy/tree/main/examples/java
*   GoogleのDifferential Privacyのライブラリのexample
    * https://github.com/google/differential-privacy/tree/main/examples/java
*   上記をOpenDPで動くように変更
    * https://docs.opendp.org/en/stable/index.html

## 変更履歴
*   2022年11月 初版
*   2023年11月 細かな修正
*   2025年12月 OpenDPに変更



## チュートリアルの全体構成
本チュートリアルは2つのPartに分かれています。
*   Part1: Differential Privacyの基本
*   Part2: 応用（複数Contributionの対応）



## 「Part1: Differential Privacyの基本」の学習内容

Part1では以下の内容について学びます。
*   データベースの1レコードの変化により、プライバシー侵害のリスク
*   差分プライバシーを満たすノイズを適用することで、上記リスクを低減できること
*   ノイズ付加による有用性の低下(元データからどの程度変化するか)
*   epsilonとsensitivity(何を隠すか)
*   Privacy Budget(プライバシーの予算)

# サンプルコード



## 準備

In [ ]:
!pip install opendp --break-system-packages # OpenDPをインストール

In [ ]:
import opendp.prelude as dp
import pandas as pd
import statistics # differential privacyを使わずに集計などをするためにインポート
import numpy as np
import matplotlib.pyplot as plt

# OpenDPの設定を有効化
dp.enable_features('contrib')


#### データを取得 (OpenMindのPyDPのチュートリアルのデータを利用)

1000行 x 6列の営業成績のデータで、各行に営業担当者のデータが保存されています。
今回は特に、5列目のsales_amountが、営業秘密としてもプライバシーとしても他人知られたくないとします。

In [ ]:
# OpenMindのPyDPのチュートリアルのデータがpublickなgithubにあるため取得
url = 'https://raw.githubusercontent.com/OpenMined/PyDP/dev/examples/Tutorial_4-Launch_demo/data/01.csv'
original_dataset = pd.read_csv(url,sep=",", engine = "python")
print("data set size: ", original_dataset.shape)
original_dataset.head()


### データセットから1行削除

ここで、仮に最初の行の営業担当者(Osbourneさん)が退職したとして、削除します。
その結果、削除前のデータセット対する集計結果と、削除後のデータセットの集計結果の差から、削除された個人のプライバシー情報が推測される恐れがあることを示したいと思います。

In [ ]:
# rupdated_datasetを、削除したデータセットとします。元のデータセットはoriginal_datasetです。
updated_dataset = original_dataset.copy()
updated_dataset = updated_dataset[1:] # 0行目を含めず1行目以降をupdated_datasetに入れているため、このデータセットには以下の行がないはずです
# Osbourne	Gillions	ogillions0@feedburner.com	31.94	Florida

In [ ]:
# まずはoriginal_datasetを確認します
original_dataset.head()

In [ ]:
# 続いてデータ1行削除したupdated_datasetを確認します
updated_dataset.head()

## 集計例1: 両方のデータセットについて合計(Sum)の差からのプライバシー侵害

元のデータセット(`original_dataset`)と、1行削除したデータセット(`updated_dataset`)について、それぞれ`sales_amount`の合計を計算します。

そして、その合計の差を計算すると、削除された行の`sales_amount`がわかってしまいます。

ここで、もし攻撃者（プライバシーに関わる情報を知りたいと考えている人）が、削除された行がOsbourneであることが知っていたとしたら(先日退職したのがOsbourne１名だけだったと知っていたとしたら)、この差の値が、Osbourneの`sales_amount`であると推測できてしまいます。

In [ ]:
# 各データセットのsales_amountのsumを計算
sum_original_dataset = sum(original_dataset['sales_amount'].to_list())
sum_updated_dataset = sum(updated_dataset['sales_amount'].to_list())
# 差を計算
sales_amount_Osbourne = sum_original_dataset - sum_updated_dataset
# floatの計算時の誤差があるため、小数点2桁目で四捨五入
sales_amount_Osbourne = round(sales_amount_Osbourne, 2)
print("sum_original_dataset - sum_updated_dataset = ", sales_amount_Osbourne)
# 一応original_datasetで一致することを確認
assert sales_amount_Osbourne == original_dataset.iloc[0, 4]

### Differential Privacyによるプライバシー侵害の抑制

このような1行の差によるプライバシー侵害が起こる可能性あるため、Differential Privacy(DP)の考え方を適用し、合計値に適切なノイズを加えます。

以降では、元のデータセット(`original_dataset`)と、1行削除したデータセット(`updated_dataset`)について適切なノイズを入れることで、ノイズ入りの合計値であると区別ができないことを示していきます。

In [ ]:
# まずは、original_datasetについて、ノイズ入りの合計値を得ます。
#
# OpenDPのTransformationとMeasurementを組み合わせてノイズ付きの合計値を計算します。
# epsilonの値、1行の最小値と最大値を指定します。
# ここでは仮に、epsilonを1.5、sales_amountの最小値を5、最大値を250と設定します。
# (補足：最小値以下のsalesの行や、最大値を超えるsalesの行はクリップ(境界値に丸められ)されます。
#       その代わり、sensitivityを小さくできます。
#       sensitivityを小さくすることによるノイズの影響の軽減と、クリップされたことによる誤差の影響のトレードオフの関係があります)
epsilon = 1.5
lower_bound = 5
upper_bound = 250

# DP計算用にデータをバウンドでクリップ（境界値内に収める）
# 注意：真の合計値(sum_original_dataset)は元データから計算済みなので、ここでクリップするのはDP用のデータのみ
original_clipped = np.clip(original_dataset['sales_amount'].values, lower_bound, upper_bound).tolist()

# 続いて、同様にupdated_datasetについても、DP計算用にデータをクリップします。
updated_clipped = np.clip(updated_dataset['sales_amount'].values, lower_bound, upper_bound).tolist()

# === OpenDPのパイプライン構築 ===
# OpenDPでは >>演算子 でデータ処理のステップを順番につなげます
# 流れ：入力空間の定義 >> データ変換(Transformation) >> ノイズ追加(Measurement)

# Step 1: 入力空間の定義
# バウンド付きのベクトルドメイン（float型、5.0～250.0の範囲）とsymmetric distanceメトリックを定義
input_space = (
    dp.vector_domain(dp.atom_domain(bounds=(lower_bound, upper_bound), T=float)),
    dp.symmetric_distance()
)

# Step 2: Transformation（データ変換、ノイズなし）
# 入力空間 >> then_sum() でベクトルを合計値に変換
# 　例：[10.0, 20.0, 30.0] → 60.0
sum_transformation = input_space >> dp.t.then_sum()

# Step 3: Measurement（ノイズ追加）
# Transformation >> then_laplace() でラプラスノイズを追加
# 　スケールパラメータ = sensitivity / epsilon = (upper_bound - lower_bound) / epsilon
# 　sensitivity = 245.0（1行の最大変化量 = upper_bound - lower_bound）
scale = float(upper_bound - lower_bound) / epsilon
sum_measurement = sum_transformation >> dp.m.then_laplace(scale=scale)

# 参考：パイプライン全体を以下のように一度に書くこともできますが、ここでは上記のように分けて書いてます。
# sum_measurement = (
#     input_space
#     >> dp.t.then_sum()
#     >> dp.m.then_laplace(scale=scale)
# )


# ノイズを入れた結果を得ます
sum_original_dataset_dp = sum_measurement(original_clipped)
sum_updated_dataset_dp = sum_measurement(updated_clipped)

# わかりやすさのため全て小数点2桁目でroundします
sum_original_dataset = round(sum_original_dataset, 2)
sum_original_dataset_dp = round(sum_original_dataset_dp, 2)
sum_updated_dataset_dp = round(sum_updated_dataset_dp, 2)

### 結果を確認

まずは、結果を見てみると、originalなノイズなしのデータと比較して、両方ともノイズが入っています。

また、ノイズが入った両方のデータは、どちらが削除したデータセットからの集計値なのか区別は困難そうです。

ノイズが入ったデータに対して差をとっても、ノイズなしのデータでの差とは異なります。

In [ ]:
print("Sum of sales_value in the orignal Dataset: {}".format(sum_original_dataset))
print("Sum of sales_value in the orignal Dataset using DP: {}".format(sum_original_dataset_dp))
print("Sum of sales_value in the updated Dataset using DP: {}".format(sum_updated_dataset_dp))

print("Difference in sum using DP: {}".format( round(sum_original_dataset_dp - sum_updated_dataset_dp, 2)))
print("Difference of actual value: {}".format(sales_amount_Osbourne))

グラフで見てもその様子がわかります。

In [ ]:
top=[('Original', sum_original_dataset), ('Updated', sum_updated_dataset), ('Original with DP', sum_original_dataset_dp), ('updated DP', sum_updated_dataset_dp)]

# originalとの差を計算したバージョン
#top=[('Original', 0), ('Updated', sum_updated_dataset - sum_original_dataset), ('Original with DP', sum_original_dataset_dp - sum_original_dataset), ('updated DP', sum_updated_dataset_dp - sum_original_dataset)]


labels, ys = zip(*top)
xs = np.arange(len(labels))
width = 0.3

plt.bar(xs, ys, width, align='center')

plt.xticks(xs, labels)
plt.yticks(ys)
plt.show()


### epsilonやsensitivityの説明

先ほどは、
` BoundedSum(epsilon= 1.5, lower_bound =  5, upper_bound = 250, dtype ='float') `
と、とりあえず設定していましたが、詳細を説明します。

epsilonは、DPのパラメータです。諸説ありますが、2.3以下が望ましいとされている文献もあります。

lower_boundとupper_boundは、sensitivityに関係します。
sensitivityは、何を隠すかに関わり、今回はある1行（ある一人）についての変化（人が増えた・減った、sales_valueが増えた・減った）を隠したいということになります。
lower_boundとupper_boundを設定することで、この変化(sensitivity)を`upper_bound - lower_bound` と設定することになります。つまり今回は、250 - 5 = 245 となります。



### 各自で試す

*   上記のコードを何度か動かし、毎回ノイズが変わること、どの程度変わるかなど確かめてください。
*   epsilonの値を変化させ、ノイズがどのように変わるか確かめてください。(例： 5, 10と増やしたり、0.1, 0.01と減らしたり)
*   同様にupper_boundやlower_boundを変えてください。



## 集計例2: 両方のデータセットについてカウント(count)の差からのプライバシー侵害

先ほどは、合計値からのプライバシー侵害でしたが、count(行数)からのプライバシー侵害の可能性もあります。もし攻撃者が、毎日行数を得れるとして、この会社の退職の時期を知りたいとしていた場合、行数が1行減っていたら退職したのだとわかります。

これを防ぐには、行数にDifferential Privacyを満たすノイズを加えることで、推測を困難にできます。

なお、countの場合は、sensitivityは1です。

In [ ]:
# まずはoriginal_datasetについて計算
# DPノイズを入れないcountを計算
count_original_dataset = len(original_dataset.id.tolist())
# 続いてupdated_datasetについても同様に計算
count_updated_dataset = len(updated_dataset.id.tolist())

# DPノイズを入れたcountを計算
# countの場合、sensitivityは1（1行の追加/削除による変化）
epsilon_count = 0.5

# === OpenDPのパイプライン構築 ===
# OpenDPでは >>演算子 でデータ処理のステップを順番につなげます
# 流れ：入力空間の定義 >> データ変換(Transformation) >> ノイズ追加(Measurement)

# Step 1: 入力空間の定義
# カウントは整数のベクトルを扱うので、T=int を指定
# boundsは指定しない（どんな整数値でも受け入れる）
input_space_count = (
    dp.vector_domain(dp.atom_domain(T=int)),
    dp.symmetric_distance()
)

# Step 2: Transformation（データ変換、ノイズなし）
# 入力空間 >> then_count() でベクトルの要素数をカウント
# 　例：[1, 2, 3, 4, 5] → 5
count_transformation = input_space_count >> dp.t.then_count()

# Step 3: Measurement（ノイズ追加）
# Transformation >> then_laplace() でラプラスノイズを追加
# 　スケールパラメータ = sensitivity / epsilon = 1 / epsilon
# 　countのsensitivity = 1（1行の追加/削除で1だけ変化）
scale_count = 1.0 / epsilon_count
count_measurement = count_transformation >> dp.m.then_laplace(scale=scale_count)

# ノイズを入れた結果を得ます
count_original_dataset_dp = int(round(count_measurement(original_dataset.id.tolist())))
count_updated_dataset_dp = int(round(count_measurement(updated_dataset.id.tolist())))

print("count_original_dataset=", count_original_dataset)
print("count_updated_dataset=", count_updated_dataset)
print("count_original_dataset_dp=", count_original_dataset_dp)
print("count_updated_dataset_dp=", count_updated_dataset_dp)



# まずはoriginal_datasetについて計算
# DPノイズを入れないcountを計算
#count_original_dataset = len(original_dataset.id.tolist())
# DPノイズを入れたcountを計算
#c_original_dataset = Count(epsilon= 0.5)
#count_original_dataset_dp = c_original_dataset.quick_result(original_dataset.id.tolist())

# 続いてupdate_datasetについても同様に計算
#count_updated_dataset = len(updated_dataset.id.tolist())
#c_updated_dataset = Count(epsilon= 0.5)
#count_updated_dataset_dp = c_updated_dataset.quick_result(updated_dataset.id.tolist())


#print("count_original_dataset=",count_original_dataset)
#print("count_updated_dataset=",count_updated_dataset)
#print("count_original_dataset_dp=",count_original_dataset_dp)
#print("count_updated_dataset_dp=",count_updated_dataset_dp)

同様に、epsilonを変化させてみると良いと思います。先ほどのSumはepsilonを1.5にしていましたので、試しに1.5にするとどうでしょうか？

# まとめ

Differential Privacyを満たすようなノイズを入れることで、個人のプライバシー侵害を防ぐことが可能

*   epsilonとsensitivityが重要
*   sumの場合はLowerBoundとUpperBoundを設定して、sensitivityが決まる
*   countの場合はsensitivityは1




# 付録：参考コード

sum集計について、DPノイズを何度か試して、ノイズがどの程度乗っているのか確かめる。


In [ ]:
# === 付録：参考コード ===
# sum集計について、DPノイズを何度か試して、ノイズがどの程度乗っているのか確かめる。

# OpenDPのパイプラインの仕組み：
# - 同じmeasurementオブジェクトを繰り返し呼び出すと、毎回新しいノイズが生成される
# - これにより、ノイズの分布（ばらつき）を観察できる
# - ノイズの大きさはepsilonとsensitivityで決まる（現在の設定：epsilon=1.5, sensitivity=245）

print("10回の実行でのノイズの変動：")
print("（真の合計値 - DP合計値 = ノイズ）\n")

for i in range(10):
    # sum_measurementパイプラインを実行
    # パイプライン： original_clipped >> then_sum() >> then_laplace() >> 結果
    dp_sum_original = sum_measurement(original_clipped)

    # 真の合計値との差を計算（これが追加されたノイズの大きさ）
    noise = sum_original_dataset - dp_sum_original
    print(f"実行 {i+1}: ノイズ = {round(noise):>6}  (DP合計 = {round(dp_sum_original, 2)})")

print(f"\n真の合計値 = {sum_original_dataset}")
print("ノイズがランダムに変化することで、元のデータが保護されます")
